# [Module 2.0] Target 데이타 준비 (Prepare Target data file) 

**[중요]: 예측의 타겟은 Store의 주별 판매 예측 입니다. Store는 Dept로 이루어져 있습니다. 문제를 간소하기 위해서 Dept의 판매금액을 모두 합산하였고, Store의 이름을 item_id 로 수정 하였습니다.
item_id는 특정 Store의 판매액으로 생각하시면 됩니다.**

* 이 노트북에서는 입력 데이타를 Forecast에 입력이 될 포맷으로 학습 (Train) 과 검증셋(Validation)으로 나누는 작업을 합니다. **자세한 과정을 알고 싶으시면 0.LookAt_RawData.ipynb 노트북 을 실행 해주세요.** 

* 이 노트북은 약 3분 정도 소요 됩니다. 
---
Data Source: https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting
    

In [1]:
import boto3
import os
import pandas as pd
import numpy as np

## 데이터 포맷 변경

**원천 데이타가 일주일의 시작이 금요일 부터 시작이 되고, Amazon Forecast는 월요일 부터 시작이 되기에 인위적으로 원본 데이타를 미래로 3일 앞으로 이동 하였습니다.**

IMPORTANT: Shift three days in the future because a weekly interval starts from Monday in Amazon Forecast.This dataset starts from Friday for an weekly interval



3일 미래로 날짜를 이동하고, Store --> item_id 로 이름 변경 합니다.

In [2]:
data_dir = 'data'
train_file_name = 'train.csv'
train_data = pd.read_csv(os.path.join(data_dir,train_file_name))
train_df = train_data.copy()
# Shift three days in the future 
train_df.Date = pd.to_datetime(train_df['Date']) + pd.DateOffset(days=3)
train_df = train_df.rename(columns={'Date':'date', 'Store':'item_id'})

item_id, date 기준으로 판매금액을 합산 합니다. 이는 Dept의 모든 판매금액을 합산하여 Dept가 사라지게 됩니다.

In [3]:
temp_df = train_df.groupby(['item_id','date'])['Weekly_Sales'].sum()
# df2 = pd.DataFrame(temp_df, columns=['Store','Date','Weekly_Sales'])
stores_sales = pd.DataFrame(temp_df)
stores_sales.reset_index(inplace=True)
# print(stores_sales.groupby('Store').agg({'Date': ['min','max']}))
stores_sales = stores_sales.set_index('date')
stores_sales.head()

,item_id,Weekly_Sales
date,,
2010-02-08,1,1643690.90
2010-02-15,1,1641957.44
2010-02-22,1,1611968.17
2010-03-01,1,1409727.59
2010-03-08,1,1554806.68


Forecast에 들어갈 수 있는 포맷으로 변경 합니다.

In [4]:

stores_sales.item_id = stores_sales.item_id.astype(str)
stores_sales.index = pd.to_datetime(stores_sales.index, format = '%Y-%m-%d' )

cols = ['Weekly_Sales', 'item_id']
stores_sales = stores_sales[cols]
stores_sales.head()

,Weekly_Sales,item_id
date,,
2010-02-08,1643690.90,1
2010-02-15,1641957.44,1
2010-02-22,1611968.17,1
2010-03-01,1409727.59,1
2010-03-08,1554806.68,1


## 사용할 데이터 기간 선택 

- 훈련 데이터는 2010.02.08 ~ 2012.07.23 기간
- 검증 데이터는 2010.07.30 ~ 2012.10.22 기간

In [5]:
# end_train_date = '2011-10-03' # Non-inclusive
# end_val_date = '2012-01-02'

end_train_date = '2012-07-30' # Non-inclusive
end_val_date = '2012-10-29'

In [6]:
target_stores_sales = stores_sales[stores_sales.index < end_train_date]
validation_stores_sales = stores_sales[stores_sales.index >= end_train_date]
validation_stores_sales = validation_stores_sales[validation_stores_sales.index < end_val_date]


In [7]:
target_stores_sales.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5805 entries, 2010-02-08 to 2012-07-23
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Weekly_Sales  5805 non-null   float64
 1   item_id       5805 non-null   object 
dtypes: float64(1), object(1)
memory usage: 136.1+ KB


In [8]:
validation_stores_sales.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 585 entries, 2012-07-30 to 2012-10-22
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Weekly_Sales  585 non-null    float64
 1   item_id       585 non-null    object 
dtypes: float64(1), object(1)
memory usage: 13.7+ KB


## Train 및 Validation Data set을 CSV 파일로 저장
두 개의 데이타 셋을 두 개의 CSV 파일로 저장하여, 추후에 Forecast에서 Data Import시에 사용 합니다.

In [9]:
# With the data in a great state, save it off as a CSV
target_time_series_filename = "target_time_series.csv"
target_time_series_path = data_dir + "/" + target_time_series_filename
target_stores_sales.to_csv(target_time_series_path, header=False)

In [10]:
# With the data in a great state, save it off as a CSV
validation_time_series_filename = "validation_time_series.csv"
validation_time_series_path = data_dir + "/" + validation_time_series_filename
validation_stores_sales.to_csv(validation_time_series_path, header=False)

다음 노트북에서 사용을 하기 위해서, 이 노트북에서 저장한 변수를 디스크에 저장을 합니다.
만약 아래의 코드에 에러가 발생을 했다고 하면, 이 노트북을 처음부터 다시 실행 해주세요.

In [11]:
%store target_time_series_filename
%store target_time_series_path
%store target_stores_sales
%store validation_time_series_filename
%store validation_time_series_path
%store validation_stores_sales

Stored 'target_time_series_filename' (str)
Stored 'target_time_series_path' (str)
Stored 'target_stores_sales' (DataFrame)
Stored 'validation_time_series_filename' (str)
Stored 'validation_time_series_path' (str)
Stored 'validation_stores_sales' (DataFrame)
